# マニュアル管理サービス開発

このノートブックは、マニュアル管理サービスの開発を行うためのノートブックです。  
開発を通して、Ruby on Rails/Vue.jsの基本的な使い方を学びます。

## セットアップ

1. `npm i && bundle install`

## 1. モデル作成

マニュアルや関連するモデルを作成します。  
最低限の情報としてタイトル、内容を持つマニュアルと、その配下に各ステップを紐づけます。

今回はRailsのscaffold機能を使って、モデル・コントローラー・ルーティングを一気に作成します。

```bash
bin/rails g scaffold manual title:string description:text --api
bin/rails g scaffold step title:string description:text manual:references --api
bin/rails db:create db:migrate
```

scaffoldコマンドで作成されるもの：
- モデルファイル（app/models/manual.rb, app/models/step.rb）
- マイグレーションファイル
- コントローラー（app/controllers/manuals_controller.rb, app/controllers/steps_controller.rb）
- ルーティング設定

モデルが作成されたかどうかを確認します。  
以下のコマンドを実行し、`Manual`モデルのレコード数が0であることを確認します。

```bash
bin/rails runner 'p Manual.count'
```

また、`Manual`モデルに`has_many`を設定し、ManualからStepを取得できるようにします。

```ruby
# app/models/manual.rb
class Manual < ApplicationRecord
  has_many :steps, dependent: :destroy
end
```

## 2. ダミーデータの作成

開発を進めるために、ダミーデータを作成します。  
今回は[faker](https://github.com/faker-ruby/faker)というGemを利用し、ダミーデータを作成します。  
まずは、Fakerをインストールします。

```sh
bundle add faker --group development,test
```

ダミーデータを作成するために、`db/seeds.rb`に以下のコードを追加します。

```ruby
# db/seeds.rb
if Manual.count.zero?
  50.times do
    Manual.create!(
      title: Faker::Book.title,
      description: Faker::Lorem.paragraph
    ).tap do |manual|
      [10, 20, 30].sample.times do
        manual.steps.create!(
          title: Faker::Book.title,
          description: Faker::Lorem.paragraph
        )
      end
    end
  end
end
```

データがない場合にのみデータを作成するようにしていることに注意してください。  
以下のコマンドを実行し、ダミーデータが作成され、`Manual`モデルのレコード数が50であることを確認します。

```
bin/rails db:seed
bin/rails runner 'p Manual.count'
```

## 3. APIの調整

scaffoldで生成されたコントローラーを`api/v1`名前空間に移動し、関連データも返すように調整します。

まず、ルーティングを整理します。  
`config/routes.rb`がscaffoldによって自動更新されているので、API用の名前空間に移動します。

```ruby
# config/routes.rb
Rails.application.routes.draw do
  namespace :api do
    namespace :v1 do
      resources :manuals
      resources :steps
    end
  end
end
```

次に、生成されたコントローラーを移動・修正します。

```bash
# コントローラーをapi/v1ディレクトリに移動
mkdir -p app/controllers/api/v1
mv app/controllers/manuals_controller.rb app/controllers/api/v1/
mv app/controllers/steps_controller.rb app/controllers/api/v1/
```

コントローラーを修正して、クラス名の修正と、レスポンスにStepを含めるようにします。

```ruby
# app/controllers/api/v1/manuals_controller.rb
class Api::V1::ManualsController < ApplicationController
  before_action :set_manual, only: %i[show update destroy]

  # GET /api/v1/manuals
  def index
    @manuals = Manual.includes(:steps)
    render json: @manuals.as_json(include: :steps) # as_jsonを追加
  end

  # GET /api/v1/manuals/:id
  def show
    render json: @manual.as_json(include: :steps) # as_jsonを追加
  end

  # 他のアクションは省略...
end
```

これで準備は完了です。  
以下のコマンドを実行し、`bin/dev`でサーバーを立ち上げ、curlやThunder Clientなどのツールを使ってAPIを叩いて動作を確認をします。

```
GET http://localhost:3001/api/v1/manuals
GET http://localhost:3001/api/v1/manuals/1
```

レスポンスにstepsが含まれていることを確認してください。

## 4. フロントエンドの作成

Vue.jsを使ったフロントエンドを作成します。  
コンポーネントはいくつか用意してあるので、一覧画面と詳細画面を作成します。

まずは、API通信用のファイルを作成します。

```typescript
// src/api/manuals.ts
const API_BASE = 'http://localhost:3000/api/v1' // ViteでProxy設定をしているため、RailsのAPIはこのURLでアクセス可能

export async function getManuals() {
  const response = await fetch(`${API_BASE}/manuals`)
  return response.json()
}

export async function getManual(id: string) {
  const response = await fetch(`${API_BASE}/manuals/${id}`)
  return response.json()
}
```

次に一覧画面を編集して、マニュアル一覧を実装していきます。
`src/views/HomeView.vue`を以下のように編集してください。

```vue
<script setup lang="ts">
import { onMounted, ref } from 'vue'
import { getManuals } from '@/api/manuals'

const manuals = ref<Manual[]>([])

onMounted(async () => {
  manuals.value = await getManuals()
})
</script>

<template>
  <v-container>
    <v-row>
      <v-col v-for="manual in manuals" :key="manual.title" cols="12">
        <v-card @click="$router.push(`/manuals/${manual.id}`)">
          <v-card-title>{{ manual.title }}</v-card-title>
          <v-card-text>{{ manual.description }}</v-card-text>
          <v-card-actions>
            <v-btn>{{ manual.steps.length }} steps</v-btn>
          </v-card-actions>
        </v-card>
      </v-col>
    </v-row>
  </v-container>
</template>
```

次は、詳細画面を作成します。  
`src/views/ManualView.vue`を以下のように編集してください。

```vue
<script setup lang="ts">
import { onMounted, ref } from 'vue'
import { useRoute } from 'vue-router'
import { getManual } from '@/api/manuals'

const manual = ref<Manual | null>(null)
const route = useRoute()

onMounted(async () => {
  const { id } = route.params
  manual.value = await getManual(id as string)
})
</script>

<template>
  <v-container>
    <div v-if="!manual">Loading...</div>
    <div v-if="manual">
      <h1>{{ manual.title }}</h1>
      <p>{{ manual.description }}</p>
      <v-row :style="{ marginTop: '20px' }">
        <v-col
          v-for="(step, index) in manual.steps"
          :key="step.title"
          cols="12"
        >
          <v-card>
            <v-card-title>STEP-{{ index + 1 }}: {{ step.title }}</v-card-title>
            <v-card-text>{{ step.description }}</v-card-text>
          </v-card>
        </v-col>
      </v-row>
    </div>
  </v-container>
</template>
```

最後に、ルーティングを設定します。  
`router`の設定を以下のように変更してください。

```typescript
// src/router/index.ts
const router = createRouter({
  history: createWebHistory(import.meta.env.BASE_URL),
  routes: [
    {
      path: '/',
      name: 'home',
      component: HomeView,
    },
    {
      path: '/manuals/:id',
      name: 'manual-view',
      component: ManualView,
    },
  ],
})
```

これで準備は完了です。  
以下のコマンドを実行し、`bin/dev`でサーバーを立ち上げ、ブラウザで[アプリケーションを開いて](http://localhost:3000)動作を確認してください。

http://localhost:3000

## 5. おまけ

### N+1問題の解消

現在の実装では、マニュアル一覧を取得する際に、各マニュアルに紐づくステップも取得しています。  
ログを見てみると、マニュアルごとにステップを取得しているため、N+1問題が発生していることが分かります。

今回は、ActiveRecordの`includes`を使って解消します。  
既に`Api::V1::ManualsController`の`index`アクションで実装済みですが、解説します。

```ruby
# N+1問題が発生するコード
def index
  @manuals = Manual.all
  # Manual.count + 1回のクエリが発行される
end

# N+1問題を解消したコード  
def index
  @manuals = Manual.includes(:steps)
  # 2回のクエリで済む（manualsとstepsを1回ずつ）
end
```

書き換え後に再度APIを叩いて、ログを確認してください。  
そうすると、マニュアル一覧のクエリとステップのクエリが1回ずつ発行されていることが分かります。

これでN+1問題が解消されました。  
このようにN+1問題はデータベースのクエリを最適化することで解消できます。

### Piniaを使った状態管理

実際のVueプロジェクトでは、サーバから取得したデータをキャッシュしておきたいといった場合にはしばしばPiniaが使われています。

```typescript
// src/stores/manuals.ts
import { defineStore } from 'pinia'
import { getManuals } from '@/api/manuals'

export const useManualStore = defineStore('manual', () => {
  const manuals = ref<Manual[]>([])
  const loading = ref(false)

  async function fetchManuals() {
    loading.value = true
    try {
      manuals.value = await getManuals()
    } finally {
      loading.value = false
    }
  }

  return { manuals, loading, fetchManuals }
})
```

これらの改善により、より堅牢で保守しやすいアプリケーションを構築できます。